In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 30.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.7 MB/s eta 0:00:00
  Created wheel for openmdao: filename=openmdao-3.35.0-py3-none-any.whl size=13158023 sha256=242d289288fb74d9573ff19c1645b4afd55c8d53569a2de40752c1f919e7eef5
  Stored in directory: /root/.cache/pip/wheels/63/25/8a/e96ea56f87876870e9b6109edafc50c070ed603729985d35ee
Successfully built openmdao


# Sellar - A Two-Discipline Problem with a Nonlinear Solver

In the monodisciplinary tutorials, we built and optimized models comprised of only a single component.
Now, we'll work through a slightly more complex problem that involves two disciplines, and hence two main components.
You'll learn how to group components together into a larger model and how to use
a [NonlinearBlockGaussSeidel](../../features/building_blocks/solvers/nonlinear_block_gs) nonlinear solver to converge a group with coupled components.

The Sellar problem is a two-discipline toy problem with each discipline described by a single equation.
There isn't any physical significance to the equations, it's just a compact example to use as a means of understanding
simple coupled models.
The output of each component feeds into the input of the other, which creates a coupled model that needs to
be converged in order for the outputs to be valid.
You can see the coupling between the two disciplines show up through the \\(y_1\\) and \\(y_2\\) variables in the following diagram that describes the problem structure: ![sellar example](images/sellar_xdsm.png)

## Building the Disciplinary Components


In the following component definitions, there is a call to [declare_partials](../../features/core_features/working_with_derivatives/approximating_partial_derivatives) in the `setup_partials` method that looks like this:

```
self.declare_partials('*', '*', method='fd')
```

This command tells OpenMDAO to approximate all the partial derivatives of that component using finite difference.
The default settings will use forward difference with an absolute step size of 1e-6, but you can change the [FD settings](../../features/core_features/working_with_derivatives/approximating_partial_derivatives) to work well for your component.


In [2]:
import numpy as np
import openmdao.api as om


class SellarDis1(om.ExplicitComponent):
    """
    Component containing Discipline 1 -- no derivatives version.
    """

    def setup(self):

        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Local Design Variable
        self.add_input('x', val=0.)

        # Coupling parameter
        self.add_input('y2', val=1.0)

        # Coupling output
        self.add_output('y1', val=1.0)

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y1 = z1**2 + z2 + x1 - 0.2*y2
        """
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        x1 = inputs['x']
        y2 = inputs['y2']

        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

In [3]:
class SellarDis2(om.ExplicitComponent):
    """
    Component containing Discipline 2 -- no derivatives version.
    """

    def setup(self):
        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Coupling parameter
        self.add_input('y1', val=1.0)

        # Coupling output
        self.add_output('y2', val=1.0)

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y2 = y1**(.5) + z1 + z2
        """

        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        y1 = inputs['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        if y1.real < 0.0:
            y1 *= -1

        outputs['y2'] = y1**.5 + z1 + z2

## Grouping and Connecting Components


We now want to build in OpenMDAO the model that is represented by the XDSM diagram above. We’ve defined the two disciplinary components, but there are still the three outputs of the model that need to be computed. Additionally, since we have the computations split up into multiple components, we need to group them all together and tell OpenMDAO how to pass data between them.

In [4]:
class SellarMDA(om.Group):
    """
    Group containing the Sellar MDA.
    """

    def setup(self):
        cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
        cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'],
                            promotes_outputs=['y1'])
        cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'],
                            promotes_outputs=['y2'])

        cycle.set_input_defaults('x', 1.0)
        cycle.set_input_defaults('z', np.array([5.0, 2.0]))

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

        self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                                  z=np.array([0.0, 0.0]), x=0.0),
                           promotes=['x', 'z', 'y1', 'y2', 'obj'])

        self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
        self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

We're working with a new class: [Group](../../features/core_features/working_with_groups/main).
This is the container that lets you build up complex model hierarchies.
Groups can contain other groups, components, or combinations of groups and components.

You can directly create instances of `Group` to work with, or you can subclass from it to define your own custom
groups. We're doing both of these things above. First, we defined our own custom `Group` subclass called `SellarMDA`.
In our run script, we created an instance of `SellarMDA` to actually run it.
Then, inside the `setup` method of `SellarMDA` we're also working directly with a `Group` instance by adding the subsystem `cycle`:


In [5]:
prob = om.Problem()
cycle = prob.model.add_subsystem('cycle', om.Group(), promotes=['*'])
cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'], promotes_outputs=['y1'])
cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'], promotes_outputs=['y2'])

# Nonlinear Block Gauss-Seidel is a gradient-free solver
cycle.nonlinear_solver = om.NonlinearBlockGS()

Start MDO with previous MDA

In [6]:
prob = om.Problem()
prob.model = SellarMDA()

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
prob.set_solver_print(level=0)

prob.run_driver()

print('minimum found at')
print(prob.get_val('x')[0])
print(prob.get_val('z'))

print('minumum objective')
print(prob.get_val('obj')[0])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.183393951729169
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------
minimum found at
0.0
[1.97763888e+00 8.83056605e-15]
minumum objective
3.183393951729169


Try changing problem parameter through interface SLSQP --> COBYLA

In [7]:
# define Sellar MDA problem
prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

# set driver and show driver options
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options

GridBox(children=(Dropdown(description='disp', options=(True, False), style=DescriptionStyle(description_width…

Output()

In [8]:
from pprint import pprint
pprint(list(prob.driver.options.items()))

[('debug_print', []),
 ('invalid_desvar_behavior', 'warn'),
 ('optimizer', 'COBYLA'),
 ('tol', 1e-06),
 ('maxiter', 200),
 ('disp', True),
 ('singular_jac_behavior', 'warn'),
 ('singular_jac_tol', 1e-16)]


In [9]:
prob.driver.recording_options

GridBox(children=(Textarea(value='[]', description='excludes', disabled=True, style=DescriptionStyle(descripti…

Output()

can change some recording option or keep them

In [10]:
pprint(list(prob.driver.recording_options.items()))

[('record_desvars', True),
 ('record_responses', False),
 ('record_objectives', True),
 ('record_constraints', True),
 ('includes', []),
 ('excludes', []),
 ('record_derivatives', False),
 ('record_inputs', True),
 ('record_outputs', True),
 ('record_residuals', False)]


In [11]:
# run the optimization
prob.setup()
prob.run_driver()


=====
cycle
=====
NL: NLBGS Converged in 8 iterations

=====
cycle
=====
NL: NLBGS Converged in 1 iterations

=====
cycle
=====
NL: NLBGS Converged in 7 iterations

=====
cycle
=====
NL: NLBGS Converged in 7 iterations

=====
cycle
=====
NL: NLBGS Converged in 8 iterations

=====
cycle
=====
NL: NLBGS Converged in 8 iterations

=====
cycle
=====
NL: NLBGS Converged in 8 iterations

=====
cycle
=====
NL: NLBGS Converged in 9 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 9 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 10 iterations

=====
cycle
=====
NL: NLBGS Converged in 9 iterations

=====
cycle
=====
NL: NLBGS Converged in 9 iterations

=====
cycle
=====
NL: NLBGS Converged in 9 iterations

===

Problem: problem3
Driver:  ScipyOptimizeDriver
  success     : True
  iterations  : 36
  runtime     : 2.8183E-01 s
  model_evals : 36
  model_time  : 1.2845E-01 s
  deriv_evals : 0
  deriv_time  : 0.0000E+00 s
  exit_status : SUCCESS